In [339]:
import numpy as np
import pandas as pd
import sklearn_crfsuite
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import metrics

# Create Evaluation Data Set 

In [498]:
df_test = pd.read_csv('../../data_v3/hr_il_tagged.csv')

df_train = pd.read_csv('../../data_v3/hr_extended_features.csv')

df_test.rename(columns={'real_activity_action': "ActivityAction"}, inplace=True)

In [500]:
df_test

,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0_x,FileName,BusinessActivity,InstanceNumber,...,IsEndBP,IsStartBP,ActivityAction,IsStartActivity,IsEndActivity,CaseId,ActivityAction,selective_filter_data,origin_request_method,origin_selective_filter_data
0,0,0,0,0,0,0,2,rec_over_http_prod_1.pcap,HrRecruitmentProcess,over,...,False,True,['Start Activity'],True,False,set(),Activity Start,version,NaN,NaN
1,1,1,1,1,1,1,57,rec_over_http_prod_1.pcap,HrRecruitmentProcess,over,...,False,False,['NoAction'],False,False,set(),NoAction,NaN,version,version
2,2,2,2,2,2,2,61,rec_over_http_prod_1.pcap,HrRecruitmentProcess,over,...,False,False,['NoAction'],False,False,set(),NoAction,NaN,NaN,NaN
3,3,3,3,3,3,3,129,rec_over_http_prod_1.pcap,HrRecruitmentProcess,over,...,False,False,['NoAction'],False,False,set(),NoAction,NaN,NaN,NaN
4,4,4,4,4,4,4,147,rec_over_http_prod_1.pcap,HrRecruitmentProcess,over,...,False,False,['NoAction'],False,False,set(),NoAction,IsNumber,authenticate,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,1293,1293,1293,1293,1293,1293,60600,rec_over_http_prod_1.pcap,HrRecruitmentProcess,over,...,False,False,['NoAction'],False,False,set(),NoAction,NaN,NaN,NaN
1294,1294,1294,1294,1294,1294,1294,61132,rec_over_http_prod_1.pcap,HrRecruitmentProcess,over,...,False,False,['NoAction'],False,False,{7},NoAction,IsNumber,execute_kw,hr.applicant_write_stage_id_8
1295,1295,1295,1295,1295,1295,1295,61136,rec_over_http_prod_1.pcap,HrRecruitmentProcess,over,...,False,False,['NoAction'],False,False,{7},NoAction,hr.applicant_archive_applicant,NaN,NaN
1296,1296,1296,1296,1296,1296,1296,61240,rec_over_http_prod_1.pcap,HrRecruitmentProcess,over,...,False,False,['NoAction'],False,False,set(),NoAction,NaN,NaN,NaN


In [386]:
df_train

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,FileName,BusinessActivity,frame.number,sniff_time_str,sniff_time,InstanceNumber,synthetic_sniff_time,...,file_data,IsStartActivity,IsEndActivity,ActivityAction,IsEndBP,IsStartBP,selective_filter_data,origin_request_method,origin_selective_filter_data,is_archive_applicant
0,0,0,0,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,26,2020-05-12 09:57:23.762710,2020-05-12 09:57:23.762710,1,2020-05-13 10:07:18.092710,...,['version'],NaN,0.0,Activity Start,False,True,version,NaN,NaN,False
1,1,1,1,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,127,2020-05-12 09:57:28.092427,2020-05-12 09:57:28.092427,1,2020-05-13 22:08:59.592427,...,"['server_version', '12.0-20190820', 'server_ve...",0.0,0.0,NoAction,False,False,NaN,version,version,False
2,2,2,2,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,135,2020-05-12 09:57:28.094494,2020-05-12 09:57:28.094494,1,2020-05-13 22:09:20.264494,...,"['authenticate', 'odoo01', 'dana.wireless@gmai...",0.0,0.0,NoAction,False,False,NaN,NaN,NaN,False
3,3,3,3,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,192,2020-05-12 09:57:28.580005,2020-05-12 09:57:28.580005,1,2020-05-13 23:30:15.860005,...,NaN,0.0,0.0,NoAction,False,False,NaN,NaN,NaN,False
4,4,4,4,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,216,2020-05-12 09:57:28.729685,2020-05-12 09:57:28.729685,1,2020-05-13 23:55:12.809685,...,['6'],0.0,0.0,NoAction,False,False,IsNumber,authenticate,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33068,33068,11958,11958,ResumeReviewActivity_250.pcap,ResumeReviewActivity,729,2020-05-14 00:17:45.346366,2020-05-14 00:17:45.346366,250,2020-05-14 04:15:16.416366,...,NaN,0.0,0.0,NoAction,False,False,NaN,NaN,NaN,False
33069,33069,11959,11959,ResumeReviewActivity_250.pcap,ResumeReviewActivity,766,2020-05-14 00:17:45.361497,2020-05-14 00:17:45.361497,250,2020-05-14 04:17:47.741497,...,['1'],0.0,0.0,NoAction,False,False,IsNumber,execute_kw,hr.applicant_write_stage_id_2,True
33070,33070,11960,11960,ResumeReviewActivity_250.pcap,ResumeReviewActivity,774,2020-05-14 00:17:45.363203,2020-05-14 00:17:45.363203,250,2020-05-14 04:18:04.803203,...,"['execute_kw', 'odoo01', '6', '123456789', 'hr...",0.0,0.0,NoAction,False,False,hr.applicant_archive_applicant,NaN,NaN,False
33071,33071,11961,11961,ResumeReviewActivity_250.pcap,ResumeReviewActivity,840,2020-05-14 00:17:45.427437,2020-05-14 00:17:45.427437,250,2020-05-14 04:28:47.207437,...,NaN,0.0,0.0,NoAction,False,False,NaN,NaN,NaN,False


In [371]:
#df_gt = pd.read_csv('../../data_v3/hr_ground_truth.csv')
#df_test["ActivityAction"]=np.select([df_test["frame.number"].isin(df_gt["start"].values),df_test["frame.number"].isin(df_gt["actual_end"].values)], ["Action Start", "Action End"],default="NoAction", )

1. Sort Values by Sniff Time
2. Validate Start and End Events after Sorting
    - Group by Instance Number and Business Activity
    - Check if frame.number is sequential 
    - Check if start activity and end activity are at beginning / end of the sequence
3. Features
    - Event with roles, request_method_call, selective_filter_data, origin_selective_filter_data
    - 

# 1 Preprocessing
1.1 Sort train set by sniff time
1.2 Form sequences
1.3 Validate sequences for ascending frame.number
1.4 Check for correct position of start and end events of each sequence

In [501]:
cols = ["event_with_roles", "request_method_call", "selective_filter_data", "origin_selective_filter_data", "InstanceNumber",
        "ActivityAction"]

df_train = df_train.sort_values(by=['sniff_time'])[cols]
df_test = df_test[cols]

In [502]:
# form sequences of length 20 
def reshape_sequence(data, seq_length):
    data = data.values
    num_seq = data.shape[0] // seq_length
    return data[:num_seq * seq_length].reshape((num_seq, seq_length, data.shape[1]))


train_seq = reshape_sequence(df_train, 20)
test_seq = reshape_sequence(df_test, 1)

In [503]:
# build feature vectors

def seq2features(seq, bw, fw, feature_list):
    return [event2features(seq, i, bw, fw, feature_list) for i in range(len(seq))]


def seq2features(seq, bw, fw, feature_list):
    # seq [0] = Position in Sequence
    # seq [0,4] = Feature 4 an Position 0 
    return [event2features(seq, i, bw, fw, feature_list) for i in range(len(seq))]


def event2features(seq, i, bw, fw, feature_list):
    # event features (position i in sequence)
    features = {
        feature_list[j]: seq[i, j] for j in range(len(feature_list))
    }

    ## backward features 1...bw

    # for k in range(1,bw+1): 
    #     index = i - k
    #     if index >= 0: 
    #         features.update({
    #             f"-{k}:{feature_list[j]}" : seq[index,j] for j in range(len(feature_list))
    #         })

    ## forward features 1...fw

    # for k in range(1, fw+1): 
    #     index = i + k
    #     if index < len(seq): 
    #         features.update({
    #             f"+{k}:{feature_list[j]}" : seq[index, j] for j in range(len(feature_list))
    #         })

    ## remove all empty String features        

    # features = {key: value for key, value in features.items() if value != ''}
    features.update({"bias": 1.0})
    return features


feature_list = [ "event_with_roles","request_method_call", "selective_file_data", "origin_file_data"]
fw = 0
bw = 0

train_features = [seq2features(s, bw, fw, feature_list) for s in train_seq]
train_labels = train_seq[:, :, -1]

test_features = [seq2features(s, bw, fw, feature_list) for s in test_seq]
test_labels = test_seq[:, :, -1]


In [508]:
# method from the paper - exclude short processes from training data 
FULL_BPS = [
    29,
    37,
    50,
    68,
    70,
    86,
    87,
    93,
    94,
    129,
    132,
    167,
    174,
    178
]

small_bp_seqs = np.array(train_features)[np.isin(train_seq[:, 1, -2], FULL_BPS)].tolist()
small_bp_labels = np.array(train_labels)[np.isin(train_seq[:, 1, -2], FULL_BPS)].tolist()

remainder_bp_seqs = np.array(train_features)[~np.isin(train_seq[:, 1, -2], FULL_BPS)].tolist()
remainder_bp_labels = np.array(train_labels)[~np.isin(train_seq[:, 1, -2], FULL_BPS)].tolist()

In [509]:
X_train, X_test, y_train, y_test = train_test_split(remainder_bp_seqs, remainder_bp_labels, test_size=0.3, random_state=42)

In [510]:
X_test.extend(small_bp_seqs)
y_test.extend(small_bp_labels)

### Train Model

In [511]:
%%time
crf = sklearn_crfsuite.CRF(
    max_iterations=200,
    c1=0.1,
    c2=0.01,
    all_possible_transitions=True
    #all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 625 ms, sys: 17.5 ms, total: 643 ms
Wall time: 769 ms


CRF(all_possible_transitions=True, c1=0.1, c2=0.01, max_iterations=200)

In [512]:
def flatten(xss):
    return [x for xs in xss for x in xs]


def evaluate(model, x, y_true):
    y_pred = model.predict(x)
    print(metrics.flat_f1_score(y_true, y_pred, average='macro', labels=model.classes_))
    print(metrics.flat_classification_report(y_true, y_pred, model.classes_))
    print(multilabel_confusion_matrix(flatten(y_true), flatten(y_pred)))


evaluate(crf, X_test, y_test)

0.9906671575932217
                precision    recall  f1-score   support

      NoAction       1.00      1.00      1.00     11953
  Activity End       0.97      0.98      0.97       310
Activity Start       1.00      1.00      1.00       337

      accuracy                           1.00     12600
     macro avg       0.99      0.99      0.99     12600
  weighted avg       1.00      1.00      1.00     12600

[[[12280    10]
  [    7   303]]

 [[12263     0]
  [    0   337]]

 [[  640     7]
  [   10 11943]]]


In [513]:
evaluate(crf, test_features, test_labels)

0.9338578547075794
                precision    recall  f1-score   support

      NoAction       0.99      1.00      1.00      1224
  Activity End       1.00      0.68      0.81        37
Activity Start       1.00      1.00      1.00        37

      accuracy                           0.99      1298
     macro avg       1.00      0.89      0.93      1298
  weighted avg       0.99      0.99      0.99      1298

[[[1261    0]
  [  12   25]]

 [[1261    0]
  [   0   37]]

 [[  62   12]
  [   0 1224]]]


In [516]:
from collections import Counter


def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))


print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common())


Top likely transitions:
Activity End -> Activity Start 3.281255
NoAction -> Activity End 1.003312
NoAction -> NoAction 0.643886
Activity End -> NoAction -1.644892
NoAction -> Activity Start -2.430615


In [520]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))


print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
13.543386 Activity End origin_file_data:hr.applicant_create
7.757604 Activity Start request_method_call:version
7.757604 Activity Start selective_file_data:version
7.409945 Activity End origin_file_data:hr.applicant_write_stage_id_6
6.931756 NoAction bias
6.256983 Activity End request_method_call:faultCode
6.256983 Activity End selective_file_data:faultCode
5.600666 Activity End origin_file_data:hr.applicant_write_stage_id_3
5.517717 Activity End request_method_call:6081
5.517717 Activity End request_method_call:5722
5.512430 NoAction request_method_call:6786
4.474187 Activity End origin_file_data:hr.applicant_write_stage_id_5
4.051562 Activity End origin_file_data:hr.applicant_write_stage_id_2
3.687969 Activity End origin_file_data:mail.activity_action_done
3.524055 Activity End request_method_call:5768
1.848610 NoAction request_method_call:id
1.165492 Activity Start event_with_roles:End Point (HR Manager)->Odoo Application: [HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\

TypeError: 'NoneType' object is not iterable